<a href="https://colab.research.google.com/github/jahaltom/Quantum-Computing/blob/main/quantum_seed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install qiskit qiskit-ibm-runtime qiskit-aer

In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService

# Replace 'MY_TOKEN' with your actual token from the IBM Quantum dashboard
QiskitRuntimeService.save_account(channel="ibm_quantum", token="199e9fd718fa0ab121194282b5108cf0e752b773a6823ddf8b9da82d4de780d0fa22c0681ec95fb8b767fcf3314921f1fb2c9f92ad580698ec9f382d21e0dda9", overwrite=True)




In [87]:
from qiskit import QuantumCircuit, transpile
from qiskit_ibm_runtime import QiskitRuntimeService, Sampler
import numpy as np

# Initialize the Qiskit Runtime service
service = QiskitRuntimeService(channel="ibm_cloud")

# Select an operational backend with at least 16 qubits
backends = service.backends(
    filters=lambda b: not b.configuration().simulator and b.status().operational and b.configuration().n_qubits >= 16
)
backend = sorted(backends, key=lambda b: b.status().pending_jobs)[0]
print(f"Using backend: {backend.name}")

def get_real_quantum_random_bits(num_bits=16):
    # Create a quantum circuit with Hadamard gates and measurements
    qc = QuantumCircuit(num_bits)
    qc.h(range(num_bits))
    qc.measure_all()

    # Transpile the circuit for the selected backend
    transpiled_circuit = transpile(qc, backend=backend, optimization_level=0)

    # Initialize the Sampler with the selected backend
    sampler = Sampler(mode=backend)

    # Run the circuit using the Sampler
    job = sampler.run([transpiled_circuit])
    result = job.result()

    # Retrieve counts from the result
    counts = result[0].data.meas.get_counts()

    # Get the most frequent bitstring
    bitstring = max(counts, key=counts.get)
    return [int(bit) for bit in bitstring.zfill(num_bits)]

# Generate quantum random bits and seed NumPy's RNG
quantum_bits = get_real_quantum_random_bits(16)
quantum_seed = int("".join(map(str, quantum_bits)), 2)

print("Quantum bits:", quantum_bits)
print("Quantum seed:", quantum_seed)

np.random.seed(quantum_seed)


<ipython-input-87-1013da17d232>:6: DeprecationWarning: The "ibm_quantum" channel option is deprecated and will be sunset on 1 July. After this date, ibm_cloud will be the only valid channel. For information on migrating to the new IBM Quantum Platform on the "ibm_cloud" channel, review the migration guide https://quantum.cloud.ibm.com/docs/migration-guides/classic-iqp-to-cloud-iqp .
  service = QiskitRuntimeService(channel="ibm_cloud")


Using backend: ibm_sherbrooke
Quantum bits: [1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0]
Quantum seed: 34646


In [95]:
from qiskit import QuantumCircuit

qc = QuantumCircuit(16)
qc.h(range(16))
qc.measure_all()

print(qc.draw(output="text"))


         ┌───┐ ░ ┌─┐                                             
    q_0: ┤ H ├─░─┤M├─────────────────────────────────────────────
         ├───┤ ░ └╥┘┌─┐                                          
    q_1: ┤ H ├─░──╫─┤M├──────────────────────────────────────────
         ├───┤ ░  ║ └╥┘┌─┐                                       
    q_2: ┤ H ├─░──╫──╫─┤M├───────────────────────────────────────
         ├───┤ ░  ║  ║ └╥┘┌─┐                                    
    q_3: ┤ H ├─░──╫──╫──╫─┤M├────────────────────────────────────
         ├───┤ ░  ║  ║  ║ └╥┘┌─┐                                 
    q_4: ┤ H ├─░──╫──╫──╫──╫─┤M├─────────────────────────────────
         ├───┤ ░  ║  ║  ║  ║ └╥┘┌─┐                              
    q_5: ┤ H ├─░──╫──╫──╫──╫──╫─┤M├──────────────────────────────
         ├───┤ ░  ║  ║  ║  ║  ║ └╥┘┌─┐                           
    q_6: ┤ H ├─░──╫──╫──╫──╫──╫──╫─┤M├───────────────────────────
         ├───┤ ░  ║  ║  ║  ║  ║  ║ └╥┘┌─┐                        
    q_7: ┤